# An Introduction to Reed-Solomon Codes

## Part 1: Polynomials and Finite Fields  
### Polynomials and Finite Fields 

The magic behind Reed-Solomon codes lies in the algebra of **polynomials over finite fields**. Before we can build the codes, we need to build our tools. This section covers the essentials of creating and working with these special mathematical objects.

### A Quick Recap of Polynomials

Let's quickly refresh some basic definitions. We'll be working with a **finite field**, which you can think of as a set of numbers where addition, subtraction, multiplication, and division are all well-defined. The simplest example is $F_p = \{0, 1, \dots, p - 1\}$ where $p$ is a prime number, and all operations are done modulo $p$.

- A **polynomial** $F(X)$ over a field $F_q$ is an expression of the form  
  $$F(X) = f_d X^d + \dots + f_1 X + f_0$$,  
  where the coefficients $f_i$ are all elements of $F_q$.

- The **degree** of the polynomial, $\text{deg}(F)$, is the highest power of $X$ with a non-zero coefficient.

- **Evaluation** means plugging in a value $\alpha \in F_q$ for $X$ to get a result $F(\alpha) \in F_q$.  
  An $\alpha$ is a **root** of $F(X)$ if $F(\alpha) = 0$.

- **Degree Mantra** :a non-zero polynomial of degree $t$ over a field can have at most $t$ distinct roots.

### Irreducibility and Field Extensions

An **irreducible polynomial** is a polynomial that cannot be factored into two non-constant polynomials of smaller degree.

For example, over $\mathbb{F}_2$, the polynomial $X^2 + X + 1$ is irreducible because its only possible factors are $X$ and $X + 1$, and neither divides it. However, $X^2 + 1$ is reducible over $\mathbb{F}_2$ since $X^2 + 1 = (X + 1)(X + 1)$.

The most powerful application of irreducible polynomials is creating **field extensions**. If we have a prime field $\mathbb{F}_p$ and an irreducible polynomial $E(X)$ of degree $s$ over that field, we can construct a new, larger field:

$$
\mathbb{F}_{p^s} \triangleq \mathbb{F}_p[X]/E(X)
$$

This new field contains all polynomials over $\mathbb{F}_p$ with degree less than $s$. This means every element looks like

$a_{s - 1} X^{s - 1} + \dots + a_1 X + a_0$.

Since each of the $s$ coefficients can be any of the $p$ values from $\mathbb{F}_p$, the new field has exactly $p^s$ **elements**.

Arithmetic in this field works as follows:

- **Addition**: Standard polynomial addition, with coefficients added modulo $p$.
- **Multiplication**: Standard polynomial multiplication, followed by taking the **remainder** of the division by the irreducible polynomial $E(X)$.

---

### Example: Constructing $\mathbb{F}_{7^2} $

Let's work over our chosen field, $\mathbb{F}_7$. The polynomial $E(X) = X^2 + 3$ is irreducible over $\mathbb{F}_7$ (you can check it has no roots in $\mathbb{F}_7$). We can use it to construct the field $\mathbb{F}_{7^2}$, which has $7^2 = 49$ elements. The elements are all linear polynomials of the form $aX + b$ where $a, b \in \mathbb{F}_7$.

Let’s take two elements in this field: $A(X) = 2X + 5$ and $B(X) = 3X + 1$.

- **Addition**: $A(X) + B(X) = (2 + 3)X + (5 + 1) = 5X + 6$.

- **Multiplication**:  
  $A(X) \cdot B(X) = (2X + 5)(3X + 1) = 6X^2 + 17X + 5$.
  
  - First, reduce coefficients mod 7: $6X^2 + 3X + 5$.
  - Now, find the remainder when dividing by $E(X) = X^2 + 3$. From $X^2 + 3 = 0$, we know $X^2 \equiv -3 \equiv 4 \pmod{E(X)}, 7$.
  - Substitute this in: $6(4) + 3X + 5 = 24 + 3X + 5 = 29 + 3X$.
  - Finally, reduce the coefficients again: $1 + 3X$.

So, in $\mathbb{F}_{7^2}$,  
$(2X + 5) \cdot (3X + 1) = 3X + 1$.

In [11]:
import numpy as np
from numpy.polynomial import polynomial as P

def poly_add_mod(p1, p2, q):
    p_add = P.polyadd(p1.coef, p2.coef)
    return P.Polynomial(p_add % q)

def poly_mul_mod(p1, p2, q):
    p_mul = P.polymul(p1.coef, p2.coef)
    return P.Polynomial(p_mul % q)

def poly_divmod_mod(p1, p2, q):
    """
    Performs polynomial division over F_q.
    Returns quotient and remainder.
    """
    # Make copies to avoid modifying original objects
    p1_coef = np.copy(p1.coef).astype(int)
    p2_coef = np.copy(p2.coef).astype(int)

    if len(p1_coef) < len(p2_coef):
        return P.Polynomial([0]), p1

    # Find multiplicative inverse of leading coefficient of divisor
    lead_inv = pow(int(p2_coef[-1]), q - 2, q)
    
    quotient = np.zeros(len(p1_coef) - len(p2_coef) + 1, dtype=int)

    while len(p1_coef) >= len(p2_coef):
        deg_diff = len(p1_coef) - len(p2_coef)
        
        coef = (p1_coef[-1] * lead_inv) % q
        quotient[deg_diff] = coef

        term = (p2_coef * coef) % q
        
        # Ensure 'term' is aligned with the part of p1_coef it's being subtracted from
        sub_len = len(p1_coef) - deg_diff
        p1_coef[-sub_len:] = (p1_coef[-sub_len:] - term) % q
        
        # Remove leading zeros from the remainder
        while len(p1_coef) > 0 and p1_coef[-1] == 0:
            p1_coef = p1_coef[:-1]

    if len(p1_coef) == 0:
        remainder_poly = P.Polynomial([0])
    else:
        remainder_poly = P.Polynomial(p1_coef)
        
    return P.Polynomial(quotient), remainder_poly


# ---- Example in GF(7^2) ----
q = 7
# E(X) = X^2 + 3, which is irreducible over F_7
E = P.Polynomial([3, 0, 1]) 

A = P.Polynomial([5, 2]) # 2X + 5
B = P.Polynomial([1, 3]) # 3X + 1

print(f"Field: F_{q}")
print(f"Irreducible Polynomial E(X): {E}")
print("-" * 20)
print(f"A(X) = {A}")
print(f"B(X) = {B}")
print("-" * 20)


# Addition in GF(7^2)
C_add = poly_add_mod(A, B, q)
print(f"A(X) + B(X) = {C_add}")

# Multiplication in GF(7^2)
C_mul_prod = poly_mul_mod(A, B, q)
_, C_mul_rem = poly_divmod_mod(C_mul_prod, E, q)

print(f"A(X) * B(X) = {C_mul_prod}")
print(f"  ... mod E(X) => {C_mul_rem}")

Field: F_7
Irreducible Polynomial E(X): 3.0 + 0.0·x + 1.0·x²
--------------------
A(X) = 5.0 + 2.0·x
B(X) = 1.0 + 3.0·x
--------------------
A(X) + B(X) = 6.0 + 5.0·x
A(X) * B(X) = 5.0 + 3.0·x + 6.0·x²
  ... mod E(X) => 1.0 + 3.0·x


### Finding an Irreducible Polynomial

To construct these fields, we need a reliable way to find an irreducible polynomial of a given degree $s$. While there are deterministic ways, a simple and effective method is a **randomized algorithm**:

1. Generate a random **monic** polynomial $F(X)$ of degree $s$ with coefficients in $\mathbb{F}_q$.
2. Test if $F(X)$ is irreducible.
3. If it is, you're done! If not, go back to step 1.

The key is the test in step 2. A polynomial $F(X)$ of degree $s$ is irreducible over $\mathbb{F}_q$ **iff** it satisfies two conditions:

1. $F(X)$ divides $X^{q^{\,s}} - X$.
2. For every prime factor $d$ of $s$, the greatest common divisor
   $\gcd\!\big(F(X),\, X^{q^{\,s/d}} - X\big)$ is $1$.

### Condition 1: $F(X)$ must divide $X^{q^{s}} - X$

This is the **"Belonging" Test**. The polynomial $X^{q^{s}} - X$ is very special: its roots are **all the elements** of the field $\mathbb{F}_{q^s}$. If $F(X)$ is genuinely an irreducible polynomial of degree $s$, its roots must live in $\mathbb{F}_{q^s}$. Therefore, $F(X)$ must be a factor of the polynomial that defines the entire field. This check confirms that $F(X)$’s roots are in the correct target field.


### Condition 2: $\gcd(F(X),\, X^{q^{\,s/d}} - X) = 1$

This is the **"Minimality" Test**. The field $\mathbb{F}_{q^s}$ contains smaller subfields, like $\mathbb{F}_{q^{s/d}}$, for every prime factor $d$ of $s$. If the $\gcd$ in this test is not $1$, it means $F(X)$ shares roots with a polynomial that defines a smaller subfield. This would imply $F(X)$ is reducible, with factors belonging to that smaller field. An irreducible polynomial of degree $s$ must be “native” to $\mathbb{F}_{q^s}$, so this test ensures its roots aren’t secretly from a smaller subfield.

**In essence:** Condition 1 checks that the roots are in the **right field**, while Condition 2 checks they aren’t in any **smaller field**. Together, they prove that $F(X)$ must be irreducible of degree $s$.

This gives us a concrete algorithm to find our building blocks.

In [12]:

def get_prime_factors(n):
    factors = set()
    d = 2
    temp_n = n
    while d * d <= temp_n:
        if temp_n % d == 0:
            factors.add(d)
            while temp_n % d == 0:
                temp_n //= d
        d += 1
    if temp_n > 1:
        factors.add(temp_n)
    return factors

def poly_gcd_mod(p1, p2, q):
    a, b = p1, p2
    while b.degree() > -1 and np.any(b.coef != 0): 
        _, r = poly_divmod_mod(a, b, q)
        a, b = b, r
    # Normalize to make it monic
    if a.degree() > -1:
        lead_inv = pow(int(a.coef[-1]), q - 2, q)
        a = P.Polynomial((a.coef * lead_inv) % q)
    return a

def poly_pow_mod(base, exp, mod_poly, q):
    res = P.Polynomial([1])
    base_rem = poly_divmod_mod(base, mod_poly, q)[1]

    while exp > 0:
        if exp % 2 == 1:
            res_prod = poly_mul_mod(res, base_rem, q)
            res = poly_divmod_mod(res_prod, mod_poly, q)[1]
        
        base_rem_sq = poly_mul_mod(base_rem, base_rem, q)
        base_rem = poly_divmod_mod(base_rem_sq, mod_poly, q)[1]
        exp //= 2
    return res

def is_irreducible(F, q, s):
    """
    Tests if a polynomial F of degree s is irreducible over F_q.
    """
    # 1. Test if F(X) divides X^(q^s) - X
    # This is equivalent to X^(q^s) = X (mod F(X))
    X = P.Polynomial([0, 1])
    # Use modular exponentiation for efficiency
    x_pow = poly_pow_mod(X, q**s, F, q)
    
    rem = poly_divmod_mod(x_pow - X, F, q)[1]
    if rem.degree() > -1 and np.any(rem.coef != 0):
        return False # F(X) does not divide X^(q^s) - X

    # 2. Test gcd condition for all prime factors of s
    prime_factors_s = get_prime_factors(s)
    for d in prime_factors_s:
        exp = q**(s // d)
        x_pow = poly_pow_mod(X, exp, F, q)
        gcd = poly_gcd_mod(F, x_pow - X, q)
        if gcd.degree() > 0:
            return False # gcd is not 1

    return True

def find_irreducible_poly(q, s):
    """Finds a random monic irreducible polynomial of degree s over F_q."""
    print(f"\nSearching for a monic irreducible polynomial of degree {s} over F_{q}...")
    while True:
        # Generate a random monic polynomial of degree s
        coeffs = np.random.randint(0, q, s)
        coeffs = np.append(coeffs, 1) # Make it monic
        F = P.Polynomial(coeffs)
        
        if is_irreducible(F, q, s):
            print(f"Found one: {F}")
            return F

# --- Example of finding a polynomial ---
find_irreducible_poly(q=7, s=2)
find_irreducible_poly(q=2, s=4);


Searching for a monic irreducible polynomial of degree 2 over F_7...
Found one: 5.0 + 3.0·x + 1.0·x²

Searching for a monic irreducible polynomial of degree 4 over F_2...
Found one: 1.0 + 0.0·x + 0.0·x² + 1.0·x³ + 1.0·x⁴
